<a href="https://colab.research.google.com/github/AlexandreXavierMartins/PST/blob/main/GA_PST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ypstruct

In [ ]:
import numpy as np
import math
import random
from ypstruct import structure
inf = math.inf


def calcula_fo(sol, tempos, n):
  fo = 0
  for i in range(1, n):
    col = sol[i-1]
    lin = sol[i]
    #print('Tarefa', lin,'depois da ', col)
    fo = fo + tempos[lin][col]
  return fo

def gera_sol(sol, n):
  v = []
  for i in range(1, n):
    v.append(i)
  #print(v)
  v = np.random.permutation(v)
  #print(v)
  for i in range(1, n):
    sol[i] = v[i-1]
  return sol

#Primeira metade do p1 e segunda de p2
def crossover(p1, p2, n):
  filho = cromossomo
  flag = n*[0]
  for i in range(n//2+1):
    filho.id[i] = p1.id[i]
    flag[p1.id[i]] = 1
  pos = n//2+1
  for i in range(1, n):
    if flag[p2.id[i]] == 0:
      filho.id[pos] = p2.id[i]
      flag[p2.id[i]] = 1
      pos = pos + 1
  return filho.deepcopy()

n = 7
tempos =  [
           [inf, inf, inf, inf, inf, inf, inf],
           [5, inf, 3, 9, 4, 2, 6],
           [3, 3, inf, 6, 2, 3, 5],
           [4, 9, 6, inf, 7, 2, 6],
           [7, 4, 2, 7, inf, 4, 8],
           [2, 2, 3, 3, 4, inf, 1], 
           [1, 6, 5, 5, 8, 1, inf]
           ]

#Parâmetro do GA
maxit = 20
npop = 20
mutation = 0.05



#Estrutura da Solução
cromossomo = structure()
cromossomo.id = None #Vetor com a solução que pode ser localizada pelo id
cromossomo.fo = None #Função objetivo de cada solução

#Geração Inicial
pop = cromossomo.repeat(npop) #Cria npop indivíduos
#Cria um indivíduo que guarda a melhor solução
bestsol = cromossomo
bestsol.id = n*[0]
bestsol.fo = np.inf
for i in range(npop):
  pop[i].id = n*[0]
  pop[i].id = gera_sol(pop[i].id, n) 
  pop[i].fo = calcula_fo(pop[i].id, tempos, n)
  if pop[i].fo < bestsol.fo:
    print('Melhorou')
    bestsol = pop[i].deepcopy()
    print(pop[i])

#LOOP
for it in range(maxit):
  #Cruzamento
  print('Geração ', it)
  popc = []
  for _ in range(npop//2):
    #Seleciona Pais
    per = np.random.permutation(npop) #Pegar os 2 primeiros
    #print(per)
    #print('Pai 1')
    #print(pop[per[0]])
    #print('Pai 2')
    #print(pop[per[1]])
    filho1 = cromossomo
    filho1 = crossover(pop[per[0]], pop[per[1]], n)
    filho2 = cromossomo
    filho2 = crossover(pop[per[1]], pop[per[0]], n)
    #print('Filho 1')
    #print(filho1)
    #print('Filho 2')
    #print(filho2)
    #break
    #Mutação para F1
    r = random.random()
    if r < mutation:
      i = random.randint(1, n-1)
      j = random.randint(1, n-1)
      while j == i:
        j = random.randint(1, n-1)
      aux = filho1.id[i]
      filho1.id[i] = filho1.id[j]
      filho1.id[j] = aux
    #Mutação para F2
    r = random.random()
    if r < mutation:
      i = random.randint(1, n-1)
      j = random.randint(1, n-1)
      while j == i:
        j = random.randint(1, n-1)
      aux = filho2.id[i]
      filho2.id[i] = filho2.id[j]
      filho2.id[j] = aux
    #Fim da mutação
    #Calcula a FO e add na população
    filho1.fo = calcula_fo(filho1.id, tempos, n)
    filho2.fo = calcula_fo(filho2.id, tempos, n)
    popc.append(filho1)
    popc.append(filho2)
    
    #print(filho1)
    #print(filho2)
    #break
    #Fim do Cruzamento e Mutação

    #Junta, Ordena e Seleciona
    pop = pop+popc
    pop = sorted(pop, key=lambda x: x.fo, reverse = False)
    pop = pop[0:npop]
    #print(pop[0])
    if pop[0].fo < bestsol.fo:
      print('Melhorou na geração', it)
      bestsol = pop[0].deepcopy()
      print(bestsol)
